In [2093]:
import numpy as np
from random import random
from functools import reduce
from collections import namedtuple
from tqdm import tqdm

# constants
PROBLEM_SIZE = 500  # dimension of the finite set U
NUMBER_SET = 1000  # number of subsets in the collection S
SETS = tuple(
    np.array([random() < 0.3 for i in range(PROBLEM_SIZE)]) for j in range(NUMBER_SET)
)  # generate sets in S
# Define State as a named tuple
State = namedtuple("State", ["taken", "cost", "heuristic"])


def goal_check(state):
    return np.all(
        reduce(np.logical_or, [SETS[i] for i in state.taken], np.zeros(PROBLEM_SIZE))
    )


assert goal_check(State(range(NUMBER_SET), 0, 0))


In [2094]:
def covered(taken):
    return np.sum(
        reduce(np.logical_or, [SETS[i] for i in taken], np.zeros(PROBLEM_SIZE))
    )


def overlap(taken):
    # spots that are coveredby just one set in take list

    matrix = np.stack([SETS[i] for i in taken], axis=-1)

    return np.sum(np.sum(matrix, axis=1) > 1)


# heuristic functions
def h1(taken):
    # number of spots covered by the union of the sets in state.taken
    return covered(taken)


def h2(taken):  # doesnmt work with current termination condition s
    # number of spots covered / number of overlapping spots
    c = covered(taken)
    o = overlap(taken)
    return c / o if o else c


In [2095]:
# successor function
def successor(state):
    return [
        State(state.taken + [i], state.cost + 1, h1(state.taken + [i]))
        for i in range(NUMBER_SET)
        if i not in state.taken
    ]


# HILL CLIMBING

```
Hill-Climbing(problem)
    current <- Make-Node(Initial-State[problem])
    loop do
        neighbor <- a highest-valued successor of current
        if Value[neighbor] <= Value[current] then return State[current]
        current <- neighbor
```


In [2096]:
def hill_climbing(init_state):
    current = init_state
    pb = tqdm()
    while True:
        neighbors = successor(current)
        next = max(neighbors, key=lambda state: state.heuristic)

        pb.update(1)
        if (
            goal_check(current)
            or next.heuristic < current.heuristic
            or len(current.taken) == NUMBER_SET
        ):
            return current

        current = next


# SIMULATED ANNELAING

```
Simulated-Annealing(problem, schedule) returns a solution state
    current <- Make-Node(Initial-State[problem])
    while True
        next <- a randomly selected successor of current
        ΔE <- Value[next] - Value[current]
        if ΔE > 0 then current <- next
        else current <- next only with probability e^(ΔE/T)
```


In [2097]:
def simulated_annealing(init_state):
    TEMP = 20

    current = init_state
    pb = tqdm()
    while True:
        pb.update(1)
        if goal_check(current) or len(current.taken) == NUMBER_SET:
            return current

        neighbors = successor(current)
        next = neighbors[np.random.randint(len(neighbors))]

        deltaE = next.heuristic - current.heuristic

        if deltaE > 0:
            current = next


# RUNS

In [2098]:
# initial state
INITIAL_STATE = State([], 0, 0)

# run hill climbing
res = hill_climbing(INITIAL_STATE)

print("Found   :", res)
print("Is sol  :", goal_check(res))
print("Overlap :", overlap(res.taken))


10it [00:00, 32.30it/s]


Found   : State(taken=[177, 810, 477, 442, 615, 142, 95, 185, 0], cost=9, heuristic=500)
Is sol  : True
Overlap : 439


In [2099]:
# run sim annealing
res = simulated_annealing(INITIAL_STATE)

print("Found   :", res)
print("Is sol  :", goal_check(res))
print("Overlap :", overlap(res.taken))


23it [00:00, 36.71it/s]

Found   : State(taken=[223, 53, 303, 384, 867, 167, 88, 753, 239, 944, 764, 192, 435, 128, 570, 78], cost=16, heuristic=500)
Is sol  : True
Overlap : 488
